*Copyright (C) 2022-23 Intel Corporation*<br>
*SPDX-License-Identifier: BSD-3-Clause*<br>
*See: https://spdx.org/licenses/*

---

# Tutorial 1: An Introduction to Graded Spikes and Fixed-point computations

**Motivation:** In this tutorial, we will discuss the basics of Lava vector algebra API and computing with graded spikes on Loihi 2. This tutorial will demonstrate simple dot-product matrix operations using graded spikes.




In [1]:
from pylab import *

Lava-VA includes a new set of processes that are compatible with Loihi 2. 

First, we can import some of the standard library using an import package. These are designed to make importing the standard libraries more simple and accessible.


In [2]:
import lava.frameworks.loihi2 as lv

Next, we'll get access to Loihi 2, or we can use the CPU backend.

In [3]:
from lava.utils import loihi

loihi.use_slurm_host(loihi_gen=loihi.ChipGeneration.N3B3)
use_loihi2 = loihi.is_installed()

if use_loihi2:
    run_cfg = lv.Loihi2HwCfg()
    print("Running on Loihi 2")
else:
    run_cfg = lv.Loihi2SimCfg(select_tag='fixed_pt')
    print("Loihi2 compiler is not available in this system. "
          "This tutorial will execute on CPU backend.")

Running on Loihi 2


Now, lets setup some inputs, and create the structure for our Loihi 2 algorithm. 

In [4]:
vec = np.array([40, 30, 20, 10])
weights = np.zeros((3,4))
weights[:, 0] = [8, 9, -7]
weights[:, 1] = [9, 8, -5]
weights[:, 2] = [8, -10, -4]
weights[:, 3] = [8, -10, -3]

# Note: we define the weights using floating points,
# this will create the equivalent fixed-point 
# representation on Loihi 2. We use the weight_exp to 
# set the dynamic range. The dynamic range is:
# weight_exp = 8 -- [-1, 1)
# weight_exp = 7 -- [-2, 2)
# weight_exp = 6 -- [-4, 4)
# ...
# weight_exp = 1 -- [-128, 128)
weights /= 10
weight_exp = 7

In [5]:
num_steps=16

In [6]:
inp_data = np.zeros((vec.shape[0], num_steps))
inp_data[:, 1] = vec.ravel()
inp_data[:, 3] = 4*vec.ravel()
inp_data[:, 5] = 16*vec.ravel()
inp_data[:, 7] = 64*vec.ravel()
inp_data[:, 9] = 256*vec.ravel()

In this case, I have created an input vector and some weights, and then I will send the input vector in with different magnitudes at different timesteps.

Next, we use the standard library to create the input layer, the synaptic weights, the neuron layer, and the readout layer.

In [7]:
invec = lv.InputVec(inp_data, loihi2=use_loihi2)

in_out_syn = lv.GradedDense(weights=weights, exp=weight_exp)

outvec = lv.GradedVec(shape=(weights.shape[0],), vth=1)

out_monitor = lv.OutputVec(shape=outvec.shape, buffer=num_steps, loihi2=use_loihi2)


There is a new interface that includes the ability to incorporate operator overloading. This allows constructions of Networks based on an algebraic syntax.



In [8]:
outvec << in_out_syn @ invec
out_monitor << outvec

Now we can run the network.

In [9]:
try:
    outvec.run(condition=lv.RunSteps(num_steps=num_steps),
              run_cfg=run_cfg)
    out_data = out_monitor.get_data()
finally:
    outvec.stop()

What we should see is the dot product of the input vector. Since we incremented the input strength, the entire vector output will also grow proportionally.

In [10]:
out_data[:,2:11:2]

array([[    83,    331,   1321,   5280,  21120],
       [    30,    119,    473,   1890,   7560],
       [   -54,   -218,   -868,  -3470, -13880]], dtype=int32)

In [11]:
weights @ vec

array([ 83.,  30., -54.])

There may be some rounding differences due to the rounding of the values, but we see the correct values compared to the numpy calculation.

## Addition operator overload

As a second example we will create two weight matrices and show how the additionn operator overload can be used.


In [12]:
# Defining two input streams
vec = np.array([40, 30, 20, 10])
weights = np.zeros((3,4))
weights[:, 0] = [8, 9, -7]
weights[:, 1] = [9, 8, -5]
weights[:, 2] = [8, -10, -4]
weights[:, 3] = [8, -10, -3]

vec2 = np.array([50, -50, 20, -20])
weights2 = np.zeros((3,4))
weights2[:, 0] = [3, -5, 4]
weights2[:, 1] = [0, -2, -10]
weights2[:, 2] = [6, 8, -4]
weights2[:, 3] = [-5, 7, -7]

weights /= 10
weights2 /= 10

weight_exp = 7


In [13]:
num_steps=16

inp_data = np.zeros((vec.shape[0], num_steps))
inp_data[:, 1] = vec.ravel()
inp_data[:, 3] = 4*vec.ravel()
inp_data[:, 5] = 16*vec.ravel()
inp_data[:, 7] = 64*vec.ravel()
inp_data[:, 9] = 256*vec.ravel()

inp_data2 = np.zeros((vec2.shape[0], num_steps))
inp_data2[:, 1] = vec2.ravel()
inp_data2[:, 3] = 4*vec2.ravel()
inp_data2[:, 5] = 16*vec2.ravel()
inp_data2[:, 7] = 64*vec2.ravel()
inp_data2[:, 9] = 256*vec2.ravel()

In [14]:
# instantiate the objects
invec1 = lv.InputVec(inp_data, loihi2=use_loihi2)
invec2 = lv.InputVec(inp_data2, loihi2=use_loihi2)

in_out_syn1 = lv.GradedDense(weights=weights, exp=weight_exp)
in_out_syn2 = lv.GradedDense(weights=weights2, exp=weight_exp)

outvec = lv.GradedVec(shape=(weights.shape[0],), vth=1)

out_monitor = lv.OutputVec(shape=outvec.shape, buffer=num_steps, loihi2=use_loihi2)


In [15]:
# compute the dot product of both input streams and add together
outvec << in_out_syn1 @ invec1 + in_out_syn2 @ invec2
out_monitor << outvec

In [16]:
try:
    outvec.run(condition=lv.RunSteps(num_steps=num_steps),
              run_cfg=run_cfg) # Loihi2SimCfg(select_tag='fixed_pt')
    out_data = out_monitor.get_data()
finally:
    outvec.stop()

In [17]:
out_data[:,2:11:2]

array([[  120,   478,  1909,  7630, 30520],
       [   17,    69,   271,  1080,  4320],
       [   22,    83,   340,  1360,  5440]], dtype=int32)

In [18]:
weights @ vec + weights2 @ vec2

array([120.,  17.,  22.])

Again we see the output results matching the numpy calculations, perhaps with some differences due to rounding.

## More algebra syntax

Another function that occurs under-the-hood is the creation of Identity connections when connecting vectors. 

This can also be supported with the addition operator.

Just have to make sure the vector shapes are correct!

In [19]:
# Defining two input streams
vec = np.array([40, 30, 20, 10])
weights = np.zeros((4,4))
weights[:, 0] = [8, 9, -7, -2]
weights[:, 1] = [9, 8, -5, 2]
weights[:, 2] = [8, -10, -4, 5]
weights[:, 3] = [8, -10, -3, -9]

vec2 = np.array([50, -50, 20, -20])
weights2 = np.zeros((4,4))
weights2[:, 0] = [3, -5, 4, -6]
weights2[:, 1] = [0, -2, -10, 0]
weights2[:, 2] = [6, 8, -4, 4]
weights2[:, 3] = [-5, 7, -7, 8]

weights3 = np.random.randint(20, size=(4,4)) - 10
weights3 = weights3 / 10

weights /= 10
weights2 /= 10

weight_exp = 7

In [20]:
num_steps=16

inp_data = np.zeros((vec.shape[0], num_steps))
inp_data[:, 1] = vec.ravel()
inp_data[:, 3] = 4*vec.ravel()
inp_data[:, 5] = 16*vec.ravel()
inp_data[:, 7] = 64*vec.ravel()
inp_data[:, 9] = 256*vec.ravel()

inp_data2 = np.zeros((vec2.shape[0], num_steps))
inp_data2[:, 1] = vec2.ravel()
inp_data2[:, 3] = 4*vec2.ravel()
inp_data2[:, 5] = 16*vec2.ravel()
inp_data2[:, 7] = 64*vec2.ravel()
inp_data2[:, 9] = 256*vec2.ravel()

In [21]:
# instantiate the objects
invec1 = lv.InputVec(inp_data, loihi2=use_loihi2)
invec2 = lv.InputVec(inp_data2, loihi2=use_loihi2)

in_out_syn1 = lv.GradedDense(weights=weights, exp=weight_exp)
in_out_syn2 = lv.GradedDense(weights=weights2, exp=weight_exp)

extra_syn = lv.GradedDense(weights=weights3, exp=weight_exp)

intvec1 = lv.GradedVec(shape=(weights.shape[0],), vth=1)
intvec2 = lv.GradedVec(shape=(weights.shape[0],), vth=1)

outvec = lv.GradedVec(shape=(weights.shape[0],), vth=1)
out_monitor = lv.OutputVec(shape=outvec.shape, buffer=num_steps, loihi2=use_loihi2)


In [22]:
intvec1 << in_out_syn1 @ invec1
intvec2 << in_out_syn2 @ invec2

outvec << intvec1 + intvec2 + extra_syn @ intvec1
out_monitor << outvec


In [23]:
try:
    outvec.run(condition=lv.RunSteps(num_steps=num_steps),
              run_cfg=run_cfg) # Loihi2SimCfg(select_tag='fixed_pt')
    out_data = out_monitor.get_data()
finally:
    outvec.stop()

In [24]:
out_data[:,3:12:2]

array([[   168,    670,   2678,  10705,  42817],
       [    -7,    -28,   -118,   -476,  -1900],
       [   123,    484,   1941,   7757,  31030],
       [   -68,   -276,  -1103,  -4415, -17659]], dtype=int32)

In [25]:
weights @ vec + weights2 @ vec2 + weights3 @ weights @ vec

array([168.8,  -7.4, 123.8, -69.6])